In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from scipy.special import gamma
from typing import ClassVar, Mapping

from ape import accounts, chain, Contract, networks, project
from kodiak_simulations_2023_07 import UniswapV3LPFixedWidthFee0Runner, UniswapV3LPFixedWidthFee1Runner

In [3]:
# SEE: https://gist.github.com/banteg/dcf6082ff7fc6ad51ce220146f29d9ff
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [4]:
# USDC/WETH 5bps pool, 8h candle fits
FILEPATH = "fits/price_0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640_12509859_17765859_2400.csv"

In [5]:
df_params = pd.read_csv(FILEPATH, index_col=0)
df_params

,0,1,2,3
norm,-0.000195,0.024510,NaN,NaN
cauchy,-0.000124,0.010094,NaN,NaN
logistic,-0.000002,0.012449,NaN,NaN
t,2.503086,0.000007,0.014541,NaN
levy_stable,1.461675,-0.006468,0.000008,0.012008


In [6]:
# optimal tick width calcs with period slider for various fits
def delta_c(alpha: float, sigma: float, tau: float) -> float:
    num = 6
    denom = (8 / (sigma * (tau ** (1/alpha))) ** 2) * (gamma(3/alpha) / gamma(1/alpha)) - 1
    return num / denom

def error_fn(alpha: float, sigma: float, tau: float, delta: float) -> float:
    z = delta / (sigma * (tau ** (1/alpha)))
    return stats.levy_stable.cdf(z, alpha, 0) - stats.levy_stable.cdf(-z, alpha, 0)

def g(alpha: float, sigma: float, tau: float, delta: float) -> float:
    return error_fn(alpha, sigma, tau, delta) / (1 - np.exp(-delta/2))

In [7]:
BLOCK_START = 12509859
BLOCK_STOP = 17765859
BLOCK_STEP = 2400

In [8]:
# TODO: different csvs for different delta choices (to compare results)
BACKTEST_FEE0_PATH = f"results/UniswapV3LPFixedWidthFee0Runner_backtest_0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640_{BLOCK_START}_{BLOCK_STOP}_{BLOCK_STEP}.csv"
BACKTEST_FEE1_PATH = f"results/UniswapV3LPFixedWidthFee1Runner_backtest_0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640_{BLOCK_START}_{BLOCK_STOP}_{BLOCK_STEP}.csv"